In [ ]:
import xlrd
import openpyxl
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import string
import os
from sklearn.impute import SimpleImputer

warnings.filterwarnings('ignore')

In [ ]:
year = input("원하는 년도 입력: ")
month_path_dir = "data/년월별/"
wea_path_dir = "총날씨데이터/"

원하는 년도 입력: 2018


In [ ]:
for i in range(1, 13):
    globals()['month_{}'.format(i)] = pd.read_csv(month_path_dir+"{}_{}.csv".format(year,i), encoding='cp949') #년월별 
    
wea = pd.read_csv(wea_path_dir+"{}_총날씨데이터.csv".format(year), encoding='cp949') #날씨

In [ ]:
import datetime

def make_datetime(arg, year): #return -> year-month-day hour:00:00
    arg['date'] = 0
    arg_d = arg['date']
    
    for i in range(0, len(arg)):
        arg_d[i] = datetime.datetime(int(year), arg['월'][i], arg['일'][i], arg['시'][i], 0, 0)
        
    return arg

In [ ]:
temp_dir = "년월별+날씨/data_wea/"
#wea = make_datetime(wea, year) #date(열) 만들기
#wea.to_csv(temp_dir+"wea_d.csv", encoding='cp949')
wea_d = pd.read_csv(temp_dir+"wea_d.csv", encoding='cp949')

# 다음부턴 불러오기

In [ ]:
nn_t = wea[wea['기온'].isnull()] #기온 열에 결측치만 존재하는 행 집합
notnn_t = wea[wea['기온'].notnull()] #기온열에 결측치가 없는 행 집헙
nn_t = nn_t.reset_index() #새로운 인덱스 
notnn_t = notnn_t.reset_index()

nn_ws = wea[wea['풍속'].isnull()] #풍속
notnn_ws = wea[wea['풍속'].notnull()] 
nn_ws = nn_ws.reset_index()
notnn_ws = notnn_ws.reset_index()

nn_precip = wea[wea['강수량'].isnull()] #강수량
notnn_precip = wea[wea['강수량'].notnull()]
nn_precip = nn_precip.reset_index()
notnn_precip = notnn_precip.reset_index()

nn_h = wea[wea['습도'].isnull()] #습도
notnn_h = wea[wea['습도'].notnull()]
nn_h = nn_h.reset_index()
notnn_h = notnn_h.reset_index()

In [ ]:
#결측치만 존재하는 데이터 프레임
nn_t = make_datetime(nn_t, year) 
nn_ws = make_datetime(nn_ws, year)
nn_precip = make_datetime(nn_precip, year)
nn_h = make_datetime(nn_h, year)

In [ ]:
#결측치가 없는 데이터 프레임
notnn_t = make_datetime(notnn_t, year)
notnn_ws = make_datetime(notnn_ws, year)
notnn_precip = make_datetime(notnn_precip, year)
notnn_h = make_datetime(notnn_h, year)

In [ ]:
# make_datetime 소요시간이 길어서 결과 csv로 저장해서 불러올 것
nn_t.to_csv(temp_dir+"nn_t.csv", encoding='cp949')
nn_ws.to_csv(temp_dir+"nn_ws.csv", encoding='cp949')
nn_precip.to_csv(temp_dir+"nn_precip.csv", encoding='cp949')
nn_h.to_csv(temp_dir+"nn_h.csv", encoding='cp949')
notnn_t.to_csv(temp_dir+"notnn_t.csv", encoding='cp949')
notnn_ws.to_csv(temp_dir+"notnn_ws.csv", encoding='cp949')
notnn_precip.to_csv(temp_dir+"notnn_precip.csv", encoding='cp949')
notnn_h.to_csv(temp_dir+"notnn_h.csv", encoding='cp949')

In [ ]:
#결측치 없음
nnt_d = pd.read_csv(temp_dir+"nn_t.csv", encoding='cp949')
nnws_d = pd.read_csv(temp_dir+"nn_ws.csv", encoding='cp949')
nnprecip_d = pd.read_csv(temp_dir+"nn_precip.csv", encoding='cp949')
nnh_d = pd.read_csv(temp_dir+"nn_h.csv", encoding='cp949')

#결측치 있음
notnnt_d = pd.read_csv(temp_dir+"notnn_t.csv", encoding='cp949')
notnnws_d = pd.read_csv(temp_dir+"notnn_ws.csv", encoding='cp949')
notnnprecip_d = pd.read_csv(temp_dir+"notnn_precip.csv", encoding='cp949')
notnnh_d = pd.read_csv(temp_dir+"notnn_h.csv", encoding='cp949')

In [ ]:
t_ja_date2 = nnt_d[['자치구','기온','date','index']]
ws_ja_date2 = nnws_d[['자치구','풍속','date','index']]
precip_ja_date2 = nnprecip_d[['자치구','강수량','date','index']]
h_ja_date2 = nnh_d[['자치구','습도','date','index']]

In [ ]:
notnnt_d = notnnt_d[['date', '기온']]
t_mean = notnnt_d.groupby('date').mean() #평균
t_mean = round(t_mean, 1) #소수점 첫째짜리에서 반올림

notnnws_d = notnnws_d[['date', '풍속']]
ws_mean = notnnws_d.groupby('date').mean()
ws_mean = round(ws_mean, 1)

notnnprecip_d = notnnprecip_d[['date', '강수량']]
precip_mean = notnnprecip_d.groupby('date').mean()
precip_mean = round(precip_mean, 1)

notnnh_d = notnnh_d[['date', '습도']]
h_mean = notnnh_d.groupby('date').mean()
h_mean = round(h_mean, 1)

In [ ]:
fi_t = pd.merge(t_ja_date2, t_mean, on=['date'], how='left') 
fi_t.drop('기온_x', axis =1, inplace=True)
fi_t.rename(columns = {'기온_y': '기온'}, inplace=True)

fi_ws = pd.merge(ws_ja_date2, ws_mean, on=['date'], how='left')
fi_ws.drop('풍속_x', axis =1, inplace=True)
fi_ws.rename(columns = {'풍속_y': '풍속'}, inplace=True)

fi_precip = pd.merge(precip_ja_date2, precip_mean, on=['date'], how='left')
fi_precip.drop('강수량_x', axis =1, inplace=True)
fi_precip.rename(columns = {'강수량_y': '강수량'}, inplace=True)

fi_h = pd.merge(h_ja_date2, h_mean, on=['date'], how='left')
fi_h.drop('습도_x', axis =1, inplace=True)
fi_h.rename(columns = {'습도_y': '습도'}, inplace=True)

In [ ]:
def fill_nan(dest, src, key):
    
    dest_len = len(dest)
    src_len = len(src)
    
    src_i = src['index']
    src_k = src[key]
    dest_k = dest[key]
    
    #dest = [dest_k[i]=src_k[j] for i in range(0, dest_len) for j in range(0, src_len) if i == src_i[j]]
   
    for i in range(0, dest_len):
        for j in range(0, src_len):
            if i == src_i[j]:
                dest_k[i] = src_k[j]
        print("{} / {}".format(i, dest_len))
        
    return dest           

In [ ]:
#하나씩 돌리는걸 권장
wea = fill_nan(wea, fi_t, '기온')
wea = fill_nan(wea, fi_ws, '풍속')
wea = fill_nan(wea, fi_precip, '강수량')
wea = fill_nan(wea, fi_h, '습도')

In [ ]:
wea.drop(['date'], axis=1, inplace=True) 

In [ ]:
#merge
for i in range(1, 13):
    globals()['month_{}'.format(i)].drop(["대여대여소명","반납대여소명","대여대여소 위도","대여대여소 경도","반납대여소 위도","반납대여소 경도"], axis=1, inplace=True) #필요없는 열 삭제
    globals()['res_{}'.format(i)] = pd.merge(left=globals()['month_{}'.format(i)], right=wea, left_on=['대여월','대여일','대여시','대여 지역'], right_on=['월','일','시','자치구'], how='left', sort=False) 

In [ ]:
#필요없는거 drop
for i in range(1, 13):
    globals()['res_{}'.format(i)].drop(["년",'월','일','시'], axis=1, inplace=True) 